In [5]:
! conda list -e > ../env.yaml

In [99]:
import pandas as pd
import cenpy
import geopandas as gpd

pd.set_option('display.max_rows', None)

In [84]:
acsVarDict = {
    'B02001_001E': 'Total population',
    'B02001_002E': 'White',
    'B02001_003E': 'Black or African American',
    'B02001_004E': 'American Indian and Alaska Native',
    'B02001_005E': 'Asian',
    'B02001_006E': 'Native Hawaiian and Other Pacific Islander',
    'B06011_001E': 'Median income'}
acsVars = ["NAME"]+list(acsVarDict.keys())

In [113]:
conn = cenpy.remote.APIConnection("ACSDT5Y2019")
acsDataBlockGroup = conn.query(acsVars, geo_unit = 'block group', geo_filter = {"state": "42","county": "101"})
acsDataTract = conn.query(acsVars, geo_unit = 'tract', geo_filter = {"state": "42","county": "101"})

In [96]:
conn.set_mapservice("tigerWMS_ACS2019")

Connection to American Community Survey: 5-Year Estimates: Detailed Tables 5-Year(ID: https://api.census.gov/data/id/ACSDT5Y2019)
With MapServer: Census ACS 2019 WMS

In [106]:
whereClause = "STATE = 42 AND COUNTY = 101"
phillyGeo = conn.mapservice.layers[10].query(where=whereClause)

/Users/zh3n/opt/miniconda3/envs/final-proj-509/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [107]:
acsDataGeo = (phillyGeo
               .merge(acsData,
                  left_on=["STATE", "COUNTY", "TRACT", "BLKGRP"],
                  right_on=["state", "county", "tract", "block group"],)
               .rename(acsVarDict,axis=1))

In [111]:
acsDataGeo = acsDataGeo[["GEOID",'Total population', 'White', 'Black or African American',
       'American Indian and Alaska Native', 'Asian',
       'Native Hawaiian and Other Pacific Islander', 'Median income']].copy()

In [6]:
acsDataGeo.head()


NameError: name 'acsDataGeo' is not defined